T5 GENERATION & EVALUATION - FINETUNED MODEL - REGULAR TEST SET

Load in test data (samples 10 and 500).
Build my own model and load in weights from 3rd epoch of finetuning training.

Generate text for 10 and then 500 samples. Use the selected hyperparameters for model.generate().
Do this in batches and then join dfs together (because it takes 6-8 hrs to run 500 samples).

Then get Rouge scores.

In [ ]:
# mount to colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q rouge_score
!pip install -q evaluate
import numpy as np
import evaluate
import rouge_score
import pandas as pd

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.0 MB/s eta 0:00:00


In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
!pip install transformers
from transformers import AutoTokenizer  , TFAutoModel
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
from pprint import pprint

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.3 MB/s eta 0:00:00


In [ ]:
# get regular test data
test_file = "drive/MyDrive/266project/billsum_test.csv"
test_data = pd.read_csv(test_file)

In [ ]:
#take a sample of 10 for testing my code, then a sample of 500 for the actual experiment
test_data_10 = test_data.sample(n=10, random_state=0)
test_data_500 = test_data.sample(n=500, random_state=0)
test_data_10 = test_data_10.reset_index(drop=True)
test_data_500 = test_data_500.reset_index(drop=True)

In [ ]:
# load out of box model
t5_model_name = 't5-base'
orig_t5_model = TFT5ForConditionalGeneration.from_pretrained(t5_model_name)
tokenizer = AutoTokenizer.from_pretrained(t5_model_name)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# get my own model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_t5_training_wrapper_model(my_t5, max_length):
    input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='input_ids')
    attention_mask = layers.Input(shape=(max_length), dtype=tf.int32, name='attention_mask')
    decoder_input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='labels')

    t5_logits = my_t5(input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids)[0]

    model = tf.keras.models.Model(inputs=[input_ids, attention_mask, decoder_input_ids],
                                  outputs=[t5_logits])
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))

    return model

In [ ]:
max_length = 168
checkpoint_dir = 'drive/MyDrive/266project/model_checkpoints/'
latest_model = 't5_billsum_weights.03.hdf5'
finetuned_model = orig_t5_model
model_wrapper = build_t5_training_wrapper_model(finetuned_model, max_length)
model_wrapper.load_weights(checkpoint_dir + latest_model)

In [ ]:
# function for generation with better HPS
def generate_candidates_HPs(orig, model):
  """Test data is a pandas df, "orig" is test_df.orig.
  Adjusted hyperparameters are used.
  """
  input = tokenizer(orig, return_tensors = 'tf',padding = True).input_ids
  output = model.generate(input,
                          max_new_tokens = 150,
                          min_length = 50,
                          num_beams = 1,
                          no_repeat_ngram_size = 2
                          )
  decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
  return decoded_output

In [ ]:
first_example = test_data_10.head(1)

In [ ]:
# Let's try with just one example. This will get us the example for the appendix and
# will also give us a better sense of how much time each row will take.
%%timeit
PROMPT = 'summarize: '
first_example['t5_finetuned_candidate'] = first_example.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, finetuned_model), axis = 1)

Token indices sequence length is longer than the specified maximum sequence length for this model (1017 > 512). Running this sequence through the model will result in indexing errors
<magic-timeit>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

12.7 s ± 73.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<magic-timeit>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
pd.set_option('display.max_colwidth', None)
first_example

Unnamed: 0  \
0        2730   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [ ]:
# create dataframes with the chunks that we will use to generate candidates
df_1 = test_data_500[:50]
df_2 = test_data_500[50:100]
df_3 = test_data_500[100:150]
df_4 = test_data_500[150:200]
df_5 = test_data_500[200:250]
df_6 = test_data_500[250:300]
df_7 = test_data_500[300:350]
df_8 = test_data_500[350:400]
df_9 = test_data_500[400:450]
df_10 = test_data_500[450:500]

In [ ]:
# for each dataframe, run generation -- 1
PROMPT = 'summarize: '
df_1['t5_finetuned_candidate'] = df_1.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, finetuned_model), axis = 1)
df_1.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_1.csv')

In [ ]:
# for each dataframe, run generation -- 2
PROMPT = 'summarize: '
df_2['t5_finetuned_candidate'] = df_2.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, finetuned_model), axis = 1)
df_2.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_2.csv')

In [ ]:
# for each dataframe, run generation -- 3
PROMPT = 'summarize: '
df_3['t5_finetuned_candidate'] = df_3.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, finetuned_model), axis = 1)
df_3.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_3.csv')

In [ ]:
# for each dataframe, run generation -- 4
PROMPT = 'summarize: '
df_4['t5_finetuned_candidate'] = df_4.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, finetuned_model), axis = 1)
df_4.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_4.csv')

In [31]:
# for each dataframe, run generation -- 5
PROMPT = 'summarize: '
df_5['t5_finetuned_candidate'] = df_5.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, finetuned_model), axis = 1)
df_5.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_5.csv')

<ipython-input-31-a8be0c786851>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['t5_finetuned_candidate'] = df_5.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, finetuned_model), axis = 1)


In [32]:
# for each dataframe, run generation -- 6
PROMPT = 'summarize: '
df_6['t5_finetuned_candidate'] = df_6.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, finetuned_model), axis = 1)
df_6.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_6.csv')

<ipython-input-32-d834b69e1986>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_6['t5_finetuned_candidate'] = df_6.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, finetuned_model), axis = 1)


In [ ]:
# for each dataframe, run generation -- 7
PROMPT = 'summarize: '
df_7['t5_finetuned_candidate'] = df_7.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, finetuned_model), axis = 1)
df_7.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_7.csv')

In [34]:
# for each dataframe, run generation -- 8
PROMPT = 'summarize: '
df_8['t5_finetuned_candidate'] = df_8.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, finetuned_model), axis = 1)
df_8.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_8.csv')

<ipython-input-34-2ec557b497d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_8['t5_finetuned_candidate'] = df_8.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, finetuned_model), axis = 1)


In [35]:
# for each dataframe, run generation -- 9
PROMPT = 'summarize: '
df_9['t5_finetuned_candidate'] = df_9.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, finetuned_model), axis = 1)
df_9.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_9.csv')

<ipython-input-35-242ebfb992fa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_9['t5_finetuned_candidate'] = df_9.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, finetuned_model), axis = 1)


In [36]:
# for each dataframe, run generation -- 10
PROMPT = 'summarize: '
df_10['t5_finetuned_candidate'] = df_10.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, finetuned_model), axis = 1)
df_10.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_10.csv')

<ipython-input-36-6c21047a4c28>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['t5_finetuned_candidate'] = df_10.apply(lambda x: generate_candidates_HPs(PROMPT + x.orig, finetuned_model), axis = 1)


In [37]:
#load all the files in
df_1_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_1.csv')
df_2_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_2.csv')
df_3_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_3.csv')
df_4_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_4.csv')
df_5_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_5.csv')
df_6_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_6.csv')
df_7_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_7.csv')
df_8_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_8.csv')
df_9_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_9.csv')
df_10_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_10.csv')

In [38]:
# concatenate all the files together
all_dfs = pd.concat([df_1_done, df_2_done, df_3_done, df_4_done, df_5_done, df_6_done, df_7_done, df_8_done, df_9_done, df_10_done], ignore_index=True)

In [39]:
all_dfs.fillna("", inplace=True)

In [40]:
len(all_dfs)

500

In [42]:
all_dfs.head(10)

Unnamed: 0.1  Unnamed: 0  \
0          0.0        2730   
1          1.0        1515   
2          2.0        2377   
3          3.0         654   
4          4.0        1998   
5          5.0         882   
6          6.0         218   
7          7.0        1412   
8          8.0         867   
9          9.0        2363   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [41]:
t5_rouge_finetuned_unprocessed = rouge.compute(predictions = all_dfs['t5_finetuned_candidate'],
                       references= all_dfs['target'])
t5_rouge_finetuned_unprocessed

{'rouge1': 0.21218709718517495,
 'rouge2': 0.07130220625646358,
 'rougeL': 0.15697291139239908,
 'rougeLsum': 0.1689604770457295}